Lo primero que hacemos es cargar todos las hojas a utilizar para posteriormente concatenarlos

In [ ]:
import pandas as pd
from pathlib import Path
from functools import reduce
import numpy as np

fecha_limite_str = '2021-07-01'  # usa ISO para evitar ambigüedad

def preparar_hoja(df: pd.DataFrame, nombre_hoja: str) -> pd.DataFrame:
    df = df.copy()
    
    # Construir FECHA_HORA
    if 'FECHA_HORA' in df.columns:
        ts = pd.to_datetime(df['FECHA_HORA'], errors='coerce')
    elif {'DIA', 'HORA'}.issubset(df.columns):

        df['HORA'] = df['HORA'].astype(str).str.extract(r'(\d{1,2}:\d{2}:\d{2})')[0]
        ts = pd.to_datetime(df['DIA'].astype(str) + ' ' + df['HORA'].astype(str), errors='coerce')
    elif 'DIA' in df.columns:
        ts = pd.to_datetime(df['DIA'], errors='coerce')
    else:
        return None

    df['FECHA_HORA'] = ts
    #quiero saber si hay nans
    print(f"Hoja {nombre_hoja}: Nulos en FECHA_HORA antes de dropna: {df['FECHA_HORA'].isna().sum()}")
    dias_nulos = df['DIA'][df['FECHA_HORA'].isna()]
    horas_nulas = df['HORA'][df['FECHA_HORA'].isna()]
    if not dias_nulos.empty or not horas_nulas.empty:
        print(f"Días nulos:\n{dias_nulos}")
        print(f"Horas nulas:\n{horas_nulas}") 
    df = df.dropna(subset=['FECHA_HORA']).reset_index(drop=True)
    # 1) Normalizar a una fila por timestamp dentro de la hoja
    #    - numéricas: 'mean' (si tus columnas son intensidades; usa 'sum' si son totales)
    #    - no numéricas: 'first'
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    other_cols = [c for c in df.columns if c not in numeric_cols + ['DIA', 'HORA', 'FECHA_HORA']]
    print(f"Duplicadas por FECHA_HORA en hoja {nombre_hoja} antes de procesmiento: {df.duplicated(subset=['FECHA_HORA']).sum()}")

    agg_map = {**{c: 'mean' for c in numeric_cols}, **{c: 'first' for c in other_cols}}
    df = (
        df.drop(columns=['DIA', 'HORA'], errors='ignore')
          .groupby('FECHA_HORA', as_index=False)
          .agg(agg_map)
    )
    print(f"Duplicadas por FECHA_HORA en hoja {nombre_hoja} despues de procesmiento: {df.duplicated(subset=['FECHA_HORA']).sum()}")
    #quiero mostrar las filas duplicadas
    filas_duplicadas = df[df.duplicated(subset=['FECHA_HORA'], keep=False)]
    if not filas_duplicadas.empty:
        print(f"Filas duplicadas por FECHA_HORA en hoja {nombre_hoja}:\n{filas_duplicadas}")
        print(filas_duplicadas)

    
    return df

# Carga y preparación por hoja
dict_de_hojas = pd.read_excel("../data\Archivos_xlsx\Planta_2020_2021.xlsx", sheet_name=None)
hojas_preparadas = [h for nombre, df in dict_de_hojas.items()
                    if (h := preparar_hoja(df, nombre)) is not None and len(h) > 0]

# Merge por FECHA_HORA (ya sin duplicados por hoja)
df_combinado = reduce(lambda l, r: pd.merge(l, r, on='FECHA_HORA', how='outer'), hojas_preparadas)
df_combinado = df_combinado.sort_values('FECHA_HORA').reset_index(drop=True)

# Diagnóstico duplicados tras el merge (debería bajar muchísimo)
print("Duplicadas exactas:", df_combinado.duplicated().sum())
print("Duplicadas por FECHA_HORA:", df_combinado.duplicated(subset=['FECHA_HORA']).sum())



# Filtro de fecha (elige 'desde' o 'hasta' explícitamente)
fecha_limite = pd.to_datetime(fecha_limite_str)

# Caso A: quedarme con datos HASTA esa fecha (incluida)
df_filtrado = df_combinado[df_combinado['FECHA_HORA'] < fecha_limite].copy()

# Caso B: quedarme con datos DESDE esa fecha (excluida)
# df_filtrado = df_combinado[df_combinado['FECHA_HORA'] > fecha_limite].copy()

print(f"Rango: {df_filtrado['FECHA_HORA'].min()} -> {df_filtrado['FECHA_HORA'].max()}")
print(f"Filas: {len(df_filtrado)}, Columnas: {df_filtrado.shape[1]}")


<>:57: SyntaxWarning: invalid escape sequence '\A'
<>:57: SyntaxWarning: invalid escape sequence '\A'
C:\Users\agusm\AppData\Local\Temp\ipykernel_30084\63391323.py:57: SyntaxWarning: invalid escape sequence '\A'
  dict_de_hojas = pd.read_excel("../data\Archivos_xlsx\Planta_2020_2021.xlsx", sheet_name=None)


Hoja Consolidado KPI: Nulos en FECHA_HORA antes de dropna: 462
Días nulos:
15573   NaT
15574   NaT
15575   NaT
15576   NaT
15577   NaT
         ..
16030   NaT
16031   NaT
16032   NaT
16033   NaT
16034   NaT
Name: DIA, Length: 462, dtype: datetime64[ns]
Horas nulas:
15573    NaN
15574    NaN
15575    NaN
15576    NaN
15577    NaN
        ... 
16030    NaN
16031    NaN
16032    NaN
16033    NaN
16034    NaN
Name: HORA, Length: 462, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado KPI antes de procesmiento: 298
Duplicadas por FECHA_HORA en hoja Consolidado KPI despues de procesmiento: 0
Hoja Consolidado Produccion: Nulos en FECHA_HORA antes de dropna: 0
Duplicadas por FECHA_HORA en hoja Consolidado Produccion antes de procesmiento: 298
Duplicadas por FECHA_HORA en hoja Consolidado Produccion despues de procesmiento: 0
Hoja Consolidado EE: Nulos en FECHA_HORA antes de dropna: 0
Duplicadas por FECHA_HORA en hoja Consolidado EE antes de procesmiento: 297
Duplicadas por FECHA_HORA 

In [2]:
dias_raw   = pd.DatetimeIndex(df_filtrado['FECHA_HORA'].dt.normalize().unique()).sort_values()
inicio = dias_raw.min()
fin    = dias_raw.max()
full   = pd.date_range(inicio, fin, freq='D')

# Faltan en el raw (no hay ningún timestamp ese día)
faltan_en_raw = full.difference(dias_raw)

print(f"Días en raw: {len(dias_raw)}")
print(f"Días desde {inicio.date()} hasta {fin.date()}: {len(full)}")
print(f"Días esperados: {len(full)}")
print(f"Faltan en raw (todas las hojas): {len(faltan_en_raw)}")
#cuales son esos dias
print(faltan_en_raw)

Días en raw: 358
Días desde 2020-07-01 hasta 2021-06-30: 365
Días esperados: 365
Faltan en raw (todas las hojas): 7
DatetimeIndex(['2020-08-29', '2020-08-30', '2020-08-31', '2020-10-31',
               '2020-12-31', '2021-03-31', '2021-05-31'],
              dtype='datetime64[ns]', freq=None)


In [3]:
df1=df_filtrado.copy()

In [4]:
df1

,FECHA_HORA,EE Planta / Hl,EE Elaboracion / Hl,EE Bodega / Hl,EE Cocina / Hl,EE Envasado / Hl,EE Linea 2 / Hl,EE Linea 3 / Hl,EE Linea 4 / Hl,EE Servicios / Hl,...,Aire Elaboracion (m3),Aire Envasado (M3),Aire Servicios (M3),Tot Aire Expulsion,Totalizador_Aire_Bodega,Totalizador_Aire_Cocina,Totalizador_Aire_L2,Totalizador_Aire_L3,Totaliador_Aire_L4,Totalizador_Aire_L5
0,2020-07-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000
1,2020-07-01 01:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000,1328.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000
2,2020-07-01 02:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1077.820,49.800,1528.380,0.000,834.530,243.29,44.450,0.12,0.220,5.010
3,2020-07-01 03:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2158.360,100.170,1725.470,0.000,1670.980,487.38,89.200,0.37,0.260,10.340
4,2020-07-01 04:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3241.720,153.700,1916.580,0.000,2509.870,731.85,135.950,0.49,1.850,15.410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8946,2021-06-30 21:00:00,9.086660,0.769001,0.760380,0.293222,1.954775,2.353923,1.797698,1.527574,5.849461,...,30427.245,18227.665,2254.090,11866.760,14019.565,4540.92,6147.755,5459.78,3772.660,2847.470
8947,2021-06-30 22:00:00,9.268396,0.777740,0.775656,0.297887,1.967890,2.371261,1.781836,1.520340,6.000492,...,31138.805,18809.365,2789.830,11886.465,14547.810,4704.53,6370.485,5613.81,3883.315,2941.755
8948,2021-06-30 23:00:00,9.318111,0.778922,0.779851,0.294093,1.954111,2.393310,1.788795,1.515841,6.062856,...,31901.425,19370.485,3305.090,11906.050,15111.015,4884.36,6594.890,5764.56,3991.720,3019.315
8949,2021-06-30 23:59:00,9.475108,0.784620,0.790020,0.298222,1.960522,2.427161,1.780074,1.514144,6.200286,...,32657.340,19939.355,3797.305,11925.535,15664.915,5066.89,6816.975,5918.17,4094.880,3109.330


In [5]:
import pandas as pd
from pathlib import Path
from functools import reduce
import numpy as np

fecha_limite_str = '2023-03-06'  # usa ISO para evitar ambigüedad

def preparar_hoja(df: pd.DataFrame, nombre_hoja: str) -> pd.DataFrame:
    df = df.copy()

    # Construir FECHA_HORA
    if 'FECHA_HORA' in df.columns:
        ts = pd.to_datetime(df['FECHA_HORA'], errors='coerce')
    elif {'DIA', 'HORA'}.issubset(df.columns):
        ts = pd.to_datetime(df['DIA'].astype(str) + ' ' + df['HORA'].astype(str), errors='coerce')
    elif 'DIA' in df.columns:
        ts = pd.to_datetime(df['DIA'], errors='coerce')
    else:
        return None

    df['FECHA_HORA'] = ts
    #quiero saber si hay nans
    print(f"Hoja {nombre_hoja}: Nulos en FECHA_HORA antes de dropna: {df['FECHA_HORA'].isna().sum()}")
    dias_nulos = df['DIA'][df['FECHA_HORA'].isna()]
    horas_nulas = df['HORA'][df['FECHA_HORA'].isna()]
    if not dias_nulos.empty or not horas_nulas.empty:
        print(f"Días nulos:\n{dias_nulos}")
        print(f"Horas nulas:\n{horas_nulas}") 
    df = df.dropna(subset=['FECHA_HORA']).reset_index(drop=True)
    # 1) Normalizar a una fila por timestamp dentro de la hoja
    #    - numéricas: 'mean' (si tus columnas son intensidades; usa 'sum' si son totales)
    #    - no numéricas: 'first'
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    other_cols = [c for c in df.columns if c not in numeric_cols + ['DIA', 'HORA', 'FECHA_HORA']]
    print(f"Duplicadas por FECHA_HORA en hoja {nombre_hoja} antes de procesmiento: {df.duplicated(subset=['FECHA_HORA']).sum()}")

    agg_map = {**{c: 'mean' for c in numeric_cols}, **{c: 'first' for c in other_cols}}
    df = (
        df.drop(columns=['DIA', 'HORA'], errors='ignore')
          .groupby('FECHA_HORA', as_index=False)
          .agg(agg_map)
    )
    print(f"Duplicadas por FECHA_HORA en hoja {nombre_hoja} despues de procesmiento: {df.duplicated(subset=['FECHA_HORA']).sum()}")
    #quiero mostrar las filas duplicadas
    filas_duplicadas = df[df.duplicated(subset=['FECHA_HORA'], keep=False)]
    if not filas_duplicadas.empty:
        print(f"Filas duplicadas por FECHA_HORA en hoja {nombre_hoja}:\n{filas_duplicadas}")
        print(filas_duplicadas)

    
    return df


# Carga y preparación por hoja
dict_de_hojas = pd.read_excel("../data\Archivos_xlsx\Planta_2021_2022.xlsx", sheet_name=None)
hojas_preparadas = [h for nombre, df in dict_de_hojas.items()
                    if (h := preparar_hoja(df, nombre)) is not None and len(h) > 0]

# Merge por FECHA_HORA (ya sin duplicados por hoja)
df_combinado = reduce(lambda l, r: pd.merge(l, r, on='FECHA_HORA', how='outer'), hojas_preparadas)
df_combinado = df_combinado.sort_values('FECHA_HORA').reset_index(drop=True)

# Diagnóstico duplicados tras el merge (debería bajar muchísimo)
print("Duplicadas exactas:", df_combinado.duplicated().sum())
print("Duplicadas por FECHA_HORA:", df_combinado.duplicated(subset=['FECHA_HORA']).sum())

# Opcional: colapsar timestamps repetidos restantes (si aún hay alguno)
df_combinado = (
    df_combinado
    .groupby('FECHA_HORA', as_index=False)   # última salvaguarda
    .agg('mean')
    .sort_values('FECHA_HORA')
    .reset_index(drop=True)
)

# Filtro de fecha (elige 'desde' o 'hasta' explícitamente)
fecha_limite = pd.to_datetime(fecha_limite_str)

# Caso A: quedarme con datos HASTA esa fecha (incluida)
df_filtrado = df_combinado[df_combinado['FECHA_HORA'] < fecha_limite].copy()

# Caso B: quedarme con datos DESDE esa fecha (excluida)
# df_filtrado = df_combinado[df_combinado['FECHA_HORA'] > fecha_limite].copy()

print(f"Rango: {df_filtrado['FECHA_HORA'].min()} -> {df_filtrado['FECHA_HORA'].max()}")
print(f"Filas: {len(df_filtrado)}, Columnas: {df_filtrado.shape[1]}")


<>:55: SyntaxWarning: invalid escape sequence '\A'
<>:55: SyntaxWarning: invalid escape sequence '\A'
C:\Users\agusm\AppData\Local\Temp\ipykernel_30084\718582386.py:55: SyntaxWarning: invalid escape sequence '\A'
  dict_de_hojas = pd.read_excel("../data\Archivos_xlsx\Planta_2021_2022.xlsx", sheet_name=None)


Hoja Consolidado KPI: Nulos en FECHA_HORA antes de dropna: 0
Duplicadas por FECHA_HORA en hoja Consolidado KPI antes de procesmiento: 299
Duplicadas por FECHA_HORA en hoja Consolidado KPI despues de procesmiento: 0
Hoja Consolidado Produccion: Nulos en FECHA_HORA antes de dropna: 1
Días nulos:
15316   NaT
Name: DIA, dtype: datetime64[ns]
Horas nulas:
15316    NaN
Name: HORA, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado Produccion antes de procesmiento: 299
Duplicadas por FECHA_HORA en hoja Consolidado Produccion despues de procesmiento: 0
Hoja Consolidado EE: Nulos en FECHA_HORA antes de dropna: 1
Días nulos:
15316   NaT
Name: DIA, dtype: datetime64[ns]
Horas nulas:
15316    NaN
Name: HORA, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado EE antes de procesmiento: 298
Duplicadas por FECHA_HORA en hoja Consolidado EE despues de procesmiento: 0
Hoja Consolidado Agua: Nulos en FECHA_HORA antes de dropna: 159
Días nulos:
496     2021-07-21
497     2021-07-21
498  

In [6]:
dias_raw   = pd.DatetimeIndex(df_filtrado['FECHA_HORA'].dt.normalize().unique()).sort_values()
inicio = dias_raw.min()
fin    = dias_raw.max()
full   = pd.date_range(inicio, fin, freq='D')

# Faltan en el raw (no hay ningún timestamp ese día)
faltan_en_raw = full.difference(dias_raw)

print(f"Días en raw: {len(dias_raw)}")
print(f"Días desde {inicio.date()} hasta {fin.date()}: {len(full)}")
print(f"Días esperados: {len(full)}")
print(f"Faltan en raw (todas las hojas): {len(faltan_en_raw)}")
#cuales son esos dias
print(faltan_en_raw)

Días en raw: 601
Días desde 2021-07-01 hasta 2023-03-05: 613
Días esperados: 613
Faltan en raw (todas las hojas): 12
DatetimeIndex(['2021-08-29', '2021-08-30', '2021-08-31', '2021-10-31',
               '2021-12-31', '2022-03-31', '2022-05-31', '2022-08-29',
               '2022-08-30', '2022-08-31', '2022-10-31', '2022-12-31'],
              dtype='datetime64[ns]', freq=None)


In [7]:
df2=df_filtrado.copy()
df2

,FECHA_HORA,EE Planta / Hl,EE Elaboracion / Hl,EE Bodega / Hl,EE Cocina / Hl,EE Envasado / Hl,EE Linea 2 / Hl,EE Linea 3 / Hl,EE Linea 4 / Hl,EE Servicios / Hl,...,Aire Elaboracion (m3),Aire Envasado (M3),Aire Servicios (M3),Tot Aire Expulsion,Totalizador_Aire_Bodega,Totalizador_Aire_Cocina,Totalizador_Aire_L2,Totalizador_Aire_L3,Totaliador_Aire_L4,Totalizador_Aire_L5
0,2021-07-01 02:00:00,0.207674,0.006979,0.015400,0.002115,0.001821,0.013610,0.013274,0.000000,0.189959,...,22361.05,16687.95,15468.00,0.00,17523.48,4837.57,6440.43,4903.54,3200.90,2143.08
1,2021-07-01 03:00:00,0.419615,0.014110,0.030876,0.003929,0.003945,0.027048,0.026549,0.000000,0.384015,...,23053.16,16897.16,15698.68,0.00,18048.57,5004.59,6529.94,4950.46,3273.15,2143.61
2,2021-07-01 04:00:00,0.625781,0.021393,0.046428,0.006044,0.005766,0.040458,0.039191,0.000000,0.573443,...,23759.91,17110.51,16011.58,0.00,18584.67,5175.24,6620.61,4997.80,3345.37,2146.73
3,2021-07-01 05:00:01,0.822242,0.027690,0.060917,0.007857,0.007586,0.053429,0.051833,0.000000,0.754146,...,24463.00,17325.05,16326.95,0.00,19118.36,5344.64,6712.13,5045.33,3417.79,2149.80
4,2021-07-01 06:00:01,1.012486,0.034593,0.075711,0.009973,0.009407,0.066299,0.063843,0.000000,0.929236,...,25172.39,17541.75,16641.86,0.00,19657.22,5515.17,6805.28,5093.20,3490.21,2153.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15004,2023-03-05 20:00:00,6.556630,0.713452,0.566726,0.305691,1.825698,2.426241,1.725187,1.615697,3.459979,...,17965.24,13438.12,16355.64,807.95,13658.30,3498.99,2494.08,5646.22,3037.55,2260.27
15005,2023-03-05 21:00:00,6.621998,0.720256,0.578699,0.308625,1.856028,2.408729,1.724542,1.594413,3.479915,...,18631.49,14123.25,17072.26,834.89,14179.85,3616.75,2681.53,5909.38,3202.69,2329.65
15006,2023-03-05 22:00:00,6.687534,0.724845,0.587880,0.311559,1.888087,2.380853,1.735978,1.579945,3.500317,...,19302.02,14818.72,17773.26,861.57,14701.79,3738.66,2878.77,6173.29,3367.22,2399.44
15007,2023-03-05 23:00:00,6.789452,0.730969,0.597780,0.314492,1.925381,2.376754,1.760276,1.574925,3.548271,...,19955.22,15496.92,18335.86,867.67,15226.81,3860.74,3061.42,6439.03,3526.59,2469.88


In [8]:
df1

,FECHA_HORA,EE Planta / Hl,EE Elaboracion / Hl,EE Bodega / Hl,EE Cocina / Hl,EE Envasado / Hl,EE Linea 2 / Hl,EE Linea 3 / Hl,EE Linea 4 / Hl,EE Servicios / Hl,...,Aire Elaboracion (m3),Aire Envasado (M3),Aire Servicios (M3),Tot Aire Expulsion,Totalizador_Aire_Bodega,Totalizador_Aire_Cocina,Totalizador_Aire_L2,Totalizador_Aire_L3,Totaliador_Aire_L4,Totalizador_Aire_L5
0,2020-07-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000,0.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000
1,2020-07-01 01:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000,1328.000,0.000,0.000,0.00,0.000,0.00,0.000,0.000
2,2020-07-01 02:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1077.820,49.800,1528.380,0.000,834.530,243.29,44.450,0.12,0.220,5.010
3,2020-07-01 03:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2158.360,100.170,1725.470,0.000,1670.980,487.38,89.200,0.37,0.260,10.340
4,2020-07-01 04:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3241.720,153.700,1916.580,0.000,2509.870,731.85,135.950,0.49,1.850,15.410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8946,2021-06-30 21:00:00,9.086660,0.769001,0.760380,0.293222,1.954775,2.353923,1.797698,1.527574,5.849461,...,30427.245,18227.665,2254.090,11866.760,14019.565,4540.92,6147.755,5459.78,3772.660,2847.470
8947,2021-06-30 22:00:00,9.268396,0.777740,0.775656,0.297887,1.967890,2.371261,1.781836,1.520340,6.000492,...,31138.805,18809.365,2789.830,11886.465,14547.810,4704.53,6370.485,5613.81,3883.315,2941.755
8948,2021-06-30 23:00:00,9.318111,0.778922,0.779851,0.294093,1.954111,2.393310,1.788795,1.515841,6.062856,...,31901.425,19370.485,3305.090,11906.050,15111.015,4884.36,6594.890,5764.56,3991.720,3019.315
8949,2021-06-30 23:59:00,9.475108,0.784620,0.790020,0.298222,1.960522,2.427161,1.780074,1.514144,6.200286,...,32657.340,19939.355,3797.305,11925.535,15664.915,5066.89,6816.975,5918.17,4094.880,3109.330


In [9]:
import pandas as pd
from pathlib import Path
from functools import reduce
import numpy as np

fecha_limite_str = '2023-03-06'  # usa ISO para evitar ambigüedad

def preparar_hoja(df: pd.DataFrame, nombre_hoja: str) -> pd.DataFrame:
    df = df.copy()
    
    # Construir FECHA_HORA
    if 'FECHA_HORA' in df.columns:
        ts = pd.to_datetime(df['FECHA_HORA'], errors='coerce')
    elif {'DIA', 'HORA'}.issubset(df.columns):

        df['HORA'] = df['HORA'].astype(str).str.extract(r'(\d{1,2}:\d{2}:\d{2})')[0]
        ts = pd.to_datetime(df['DIA'].astype(str) + ' ' + df['HORA'].astype(str), errors='coerce')
    elif 'DIA' in df.columns:
        ts = pd.to_datetime(df['DIA'], errors='coerce')
    else:
        return None

    df['FECHA_HORA'] = ts
    #quiero saber si hay nans
    print(f"Hoja {nombre_hoja}: Nulos en FECHA_HORA antes de dropna: {df['FECHA_HORA'].isna().sum()}")
    dias_nulos = df['DIA'][df['FECHA_HORA'].isna()]
    horas_nulas = df['HORA'][df['FECHA_HORA'].isna()]
    if not dias_nulos.empty or not horas_nulas.empty:
        print(f"Días nulos:\n{dias_nulos}")
        print(f"Horas nulas:\n{horas_nulas}") 
    df = df.dropna(subset=['FECHA_HORA']).reset_index(drop=True)
    # 1) Normalizar a una fila por timestamp dentro de la hoja
    #    - numéricas: 'mean' (si tus columnas son intensidades; usa 'sum' si son totales)
    #    - no numéricas: 'first'
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    other_cols = [c for c in df.columns if c not in numeric_cols + ['DIA', 'HORA', 'FECHA_HORA']]
    print(f"Duplicadas por FECHA_HORA en hoja {nombre_hoja} antes de procesmiento: {df.duplicated(subset=['FECHA_HORA']).sum()}")

    agg_map = {**{c: 'mean' for c in numeric_cols}, **{c: 'first' for c in other_cols}}
    df = (
        df.drop(columns=['DIA', 'HORA'], errors='ignore')
          .groupby('FECHA_HORA', as_index=False)
          .agg(agg_map)
    )
    print(f"Duplicadas por FECHA_HORA en hoja {nombre_hoja} despues de procesmiento: {df.duplicated(subset=['FECHA_HORA']).sum()}")
    #quiero mostrar las filas duplicadas
    filas_duplicadas = df[df.duplicated(subset=['FECHA_HORA'], keep=False)]
    if not filas_duplicadas.empty:
        print(f"Filas duplicadas por FECHA_HORA en hoja {nombre_hoja}:\n{filas_duplicadas}")
        print(filas_duplicadas)

    
    return df

# Carga y preparación por hoja
dict_de_hojas = pd.read_excel("../data\Archivos_xlsx\Planta_2023.xlsx", sheet_name=None)
#Solo quiero seleccionar las hojas cuyo nombre empiece con "Consolidado"
dict_de_hojas = {nombre: df for nombre, df in dict_de_hojas.items() if nombre.startswith("Consolidado")}
hojas_preparadas = [h for nombre, df in dict_de_hojas.items()
                    if (h := preparar_hoja(df, nombre)) is not None and len(h) > 0]

# Merge por FECHA_HORA (ya sin duplicados por hoja)
df_combinado = reduce(lambda l, r: pd.merge(l, r, on='FECHA_HORA', how='outer'), hojas_preparadas)
df_combinado = df_combinado.sort_values('FECHA_HORA').reset_index(drop=True)

# Diagnóstico duplicados tras el merge (debería bajar muchísimo)
print("Duplicadas exactas:", df_combinado.duplicated().sum())
print("Duplicadas por FECHA_HORA:", df_combinado.duplicated(subset=['FECHA_HORA']).sum())



# Filtro de fecha (elige 'desde' o 'hasta' explícitamente)
fecha_limite = pd.to_datetime(fecha_limite_str)

# Caso A: quedarme con datos HASTA esa fecha (incluida)
df_filtrado = df_combinado[df_combinado['FECHA_HORA'] >= fecha_limite].copy()

# Caso B: quedarme con datos DESDE esa fecha (excluida)
# df_filtrado = df_combinado[df_combinado['FECHA_HORA'] > fecha_limite].copy()

print(f"Rango: {df_filtrado['FECHA_HORA'].min()} -> {df_filtrado['FECHA_HORA'].max()}")
print(f"Filas: {len(df_filtrado)}, Columnas: {df_filtrado.shape[1]}")
dias_raw   = pd.DatetimeIndex(df_filtrado['FECHA_HORA'].dt.normalize().unique()).sort_values()
inicio = dias_raw.min()
fin    = dias_raw.max()
full   = pd.date_range(inicio, fin, freq='D')

# Faltan en el raw (no hay ningún timestamp ese día)
faltan_en_raw = full.difference(dias_raw)

print(f"Días en raw: {len(dias_raw)}")
print(f"Días desde {inicio.date()} hasta {fin.date()}: {len(full)}")
print(f"Días esperados: {len(full)}")
print(f"Faltan en raw (todas las hojas): {len(faltan_en_raw)}")
#cuales son esos dias
print(faltan_en_raw)
df3=df_filtrado.copy()

<>:56: SyntaxWarning: invalid escape sequence '\A'
<>:56: SyntaxWarning: invalid escape sequence '\A'
C:\Users\agusm\AppData\Local\Temp\ipykernel_30084\2486818730.py:56: SyntaxWarning: invalid escape sequence '\A'
  dict_de_hojas = pd.read_excel("../data\Archivos_xlsx\Planta_2023.xlsx", sheet_name=None)


Hoja Consolidado KPI: Nulos en FECHA_HORA antes de dropna: 1
Días nulos:
12009   NaT
Name: DIA, dtype: datetime64[ns]
Horas nulas:
12009    NaN
Name: HORA, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado KPI antes de procesmiento: 199
Duplicadas por FECHA_HORA en hoja Consolidado KPI despues de procesmiento: 0
Hoja Consolidado Produccion: Nulos en FECHA_HORA antes de dropna: 1
Días nulos:
12009   NaT
Name: DIA, dtype: datetime64[ns]
Horas nulas:
12009    NaN
Name: HORA, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado Produccion antes de procesmiento: 199
Duplicadas por FECHA_HORA en hoja Consolidado Produccion despues de procesmiento: 0
Hoja Consolidado EE: Nulos en FECHA_HORA antes de dropna: 2
Días nulos:
12009   NaT
12010   NaT
Name: DIA, dtype: datetime64[ns]
Horas nulas:
12009    NaN
12010    NaN
Name: HORA, dtype: object
Duplicadas por FECHA_HORA en hoja Consolidado EE antes de procesmiento: 195
Duplicadas por FECHA_HORA en hoja Consolidado EE despues de p

In [10]:
vector_df=[df1,df2,df3]
df_concatenado = pd.concat(vector_df, ignore_index=True)


In [11]:
df_concatenado

,FECHA_HORA,EE Planta / Hl,EE Elaboracion / Hl,EE Bodega / Hl,EE Cocina / Hl,EE Envasado / Hl,EE Linea 2 / Hl,EE Linea 3 / Hl,EE Linea 4 / Hl,EE Servicios / Hl,...,Unnamed: 18,Agua Paste L3,Agua Lavadora L3,Agua Cond Evaporativos,Agua Calderas,Agua Efluentes,Agua CO2,Tot Vap Paste L3 / Hora,Tot Vap Lav L3 / Hora,Medicion Gas Planta (M3)
0,2020-07-01 00:00:00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-01 01:00:00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-01 02:00:00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-07-01 03:00:00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-07-01 04:00:00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29704,2023-10-26 03:00:00,19.366571,2.040100,2.325815,NaN,3.258145,2.316117,NaN,NaN,11.563910,...,580.0,0.0,0.000000,56.992188,3.300293,4.200195,8.601562,0.0,0.0,384.0
29705,2023-10-26 04:00:00,20.934877,2.161290,2.455645,NaN,3.431452,2.446269,NaN,NaN,12.657258,...,628.0,0.0,0.000000,75.000000,4.899902,4.700195,11.400391,0.0,0.0,506.0
29706,2023-10-26 05:00:00,20.861716,2.095390,2.391097,NaN,3.405405,2.429268,NaN,NaN,12.724960,...,628.0,0.0,0.000000,94.003906,5.600098,5.500000,14.201172,0.0,0.0,639.0
29707,2023-10-26 06:00:00,20.964980,2.021419,2.321285,NaN,3.394913,2.432289,NaN,NaN,12.878179,...,628.0,0.0,0.000000,111.992188,6.300293,5.899902,17.000000,0.0,0.0,771.0


In [12]:
prefijos = [
    "Consolidado Agua__",
    "Consolidado EE__",
    "Consolidado KPI__",
    "Consolidado Produccion__",
    "Consolidado GasVapor__",
    "Consolidado Aire__"
]

for p in prefijos:
    df_concatenado.columns = df_concatenado.columns.str.replace(p, "", regex=False)

print(df_concatenado.columns.tolist())

['FECHA_HORA', 'EE Planta / Hl', 'EE Elaboracion / Hl', 'EE Bodega / Hl', 'EE Cocina / Hl', 'EE Envasado / Hl', 'EE Linea 2 / Hl', 'EE Linea 3 / Hl', 'EE Linea 4 / Hl', 'EE Servicios / Hl', 'EE Sala Maq / Hl', 'EE Frio / Hl', 'EE Aire / Hl', 'EE CO2 / Hl', 'EE Caldera / Hl', 'EE Eflu / Hl', 'EE Agua / Hl', 'EE Resto Serv / Hl', 'EE Resto Planta / Hl', 'Agua Planta / Hl', 'Agua Elab / Hl', 'Agua Bodega / Hl', 'Agua Cocina / Hl', 'Agua Envas / Hl', 'Agua Linea 2/Hl', 'Agua Linea 3/Hl', 'Agua Linea 4/Hl', 'Agua Linea 5/Hl', 'Agua Servicios/Hl', 'Agua Planta de Agua/Hl', 'Produccion Agua / Hl', 'ET Planta / Hl', 'ET Elab/Hl', 'ET Bodega/Hl', 'ET Cocina/Hl', 'ET Envasado/Hl', 'ET Linea 2/Hl', 'ET Linea 3/Hl', 'ET Linea 4/Hl', 'ET Linea 5/Hl', 'ET Servicios / Hl', 'Aire Planta / Hl', 'Aire Elaboracion / Hl', 'Aire Cocina / Hl', 'Aire Bodega / Hl', 'Aire Envasado / Hl', 'Aire L2 / Hl', 'Aire L3 / Hl', 'Aire L4 / Hl', 'Aire L5 / Hl', 'Aire Servicios / Hl', 'CO 2 / Hl', 'CO 2 Filtro / Hl', 'CO 

In [13]:
import pandas as pd

# Agrupa por la fecha (la parte de día de 'FECHA_HORA') y encuentra la hora máxima (idxmax) para esa fecha.
indices_ultima_medicion = df_concatenado.groupby(
    df_concatenado['FECHA_HORA'].dt.date
)['FECHA_HORA'].idxmax()

# --- PASO 3: Filtrar el DataFrame Original ---
# Selecciona solo las filas correspondientes a los índices de la última medición.
df_final = df_concatenado.loc[indices_ultima_medicion]



# Muestra el resultado
print("✅ Proceso completado. El DataFrame final está listo.")
print(df_final.head())
print(f"Dimensiones del DataFrame final: {df_final.shape}")
# Días presentes en cada etapa
dias_final = pd.DatetimeIndex(df_final['FECHA_HORA'].dt.normalize().unique()).sort_values()
dias_raw   = pd.DatetimeIndex(df_concatenado['FECHA_HORA'].dt.normalize().unique()).sort_values()

# Rango común para evaluar
inicio = max(dias_final.min(), dias_raw.min())
fin    = min(dias_final.max(), dias_raw.max())
full   = pd.date_range(inicio, fin, freq='D')

# Faltan en el raw (no hay ningún timestamp ese día)
faltan_en_raw = full.difference(dias_raw)

# Días que estaban en raw pero se perdieron al construir df_final
perdidos_en_proceso = dias_raw.difference(dias_final)

print(f"Días esperados: {len(full)}")
print(f"Dias en df_final: {len(dias_final)}")
print(f"Va desde {dias_final.min().date()} hasta {dias_final.max().date()}")

✅ Proceso completado. El DataFrame final está listo.
             FECHA_HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  \
24  2020-07-01 23:59:00      642.727209            47.145349       69.023256   
49  2020-07-02 23:59:00        7.767254             0.769609        0.798838   
74  2020-07-03 23:59:00        8.801205             0.862593        0.835762   
99  2020-07-04 23:59:00        5.175639             0.439225        0.371077   
124 2020-07-05 23:59:00        7.924665             0.802365        0.717787   

     EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  \
24         0.000000         13.813953        14.578784         0.000000   
49         0.319229          2.358593         4.158962         1.506838   
74         0.260924          1.985462        39.076667         1.448962   
99         0.258048          1.442114         4.348182         1.355238   
124        0.301592          1.664726         5.125920         2.704348   

     EE Linea 4

In [14]:
import pandas as pd
import io 

# --- 2. Extraer Características (¡sin la hora!) ---

# 'año' es una tendencia, no un ciclo

# Características cíclicas
df_final['mes'] = df_final['FECHA_HORA'].dt.month
df_final['dia_semana'] = df_final['FECHA_HORA'].dt.dayofweek  # 0=Lunes, 6=Domingo

# --- 3. Transformar Cíclicas (Seno/Coseno) ---

# Mes (1-12)
df_final['mes_sin'] = np.sin(2 * np.pi * df_final['mes'] / 12)
df_final['mes_cos'] = np.cos(2 * np.pi * df_final['mes'] / 12)

# Día de la semana (0-6)
df_final['dia_semana_sin'] = np.sin(2 * np.pi * df_final['dia_semana'] / 7)
df_final['dia_semana_cos'] = np.cos(2 * np.pi * df_final['dia_semana'] / 7)

# --- 4. Limpieza Final ---
# Eliminamos la columna original y las intermedias
columnas_a_eliminar = ['mes', 'dia_semana'] #mas adelante vamos a elimnar la FECHA_HORA por ahora lo dejamos para tener una mejor trazabilidad de los datos
df_final = df_final.drop(columns=columnas_a_eliminar)

print("DataFrame 100% numérico y listo para escalar:")
print(df_final.head())

def descargar_dataframe_como_csv(df, nombre_archivo="datos_descargados.csv"):
    """
    Guarda un DataFrame de Pandas como un archivo CSV y 
    fuerza la descarga en el navegador si se ejecuta en Colab o Jupyter.
    """
    try:
        # 1. Guardar el archivo en el disco local del entorno
        df.to_csv(nombre_archivo, index=False, encoding='utf-8')
        print(f"✅ DataFrame guardado exitosamente como '{nombre_archivo}' en el entorno local.")

        # 2. Lógica para forzar la descarga al ordenador del usuario
        # ---
        


    except Exception as e:
        print(f"❌ Ocurrió un error al guardar o descargar el archivo: {e}")

df_final=pd.DataFrame(df_final)

descargar_dataframe_como_csv(df_final, "../data\processed/foundational_dataset.csv")

DataFrame 100% numérico y listo para escalar:
             FECHA_HORA  EE Planta / Hl  EE Elaboracion / Hl  EE Bodega / Hl  \
24  2020-07-01 23:59:00      642.727209            47.145349       69.023256   
49  2020-07-02 23:59:00        7.767254             0.769609        0.798838   
74  2020-07-03 23:59:00        8.801205             0.862593        0.835762   
99  2020-07-04 23:59:00        5.175639             0.439225        0.371077   
124 2020-07-05 23:59:00        7.924665             0.802365        0.717787   

     EE Cocina / Hl  EE Envasado / Hl  EE Linea 2 / Hl  EE Linea 3 / Hl  \
24         0.000000         13.813953        14.578784         0.000000   
49         0.319229          2.358593         4.158962         1.506838   
74         0.260924          1.985462        39.076667         1.448962   
99         0.258048          1.442114         4.348182         1.355238   
124        0.301592          1.664726         5.125920         2.704348   

     EE Linea 4 / Hl  

<>:50: SyntaxWarning: invalid escape sequence '\p'
<>:50: SyntaxWarning: invalid escape sequence '\p'
C:\Users\agusm\AppData\Local\Temp\ipykernel_30084\3600581868.py:50: SyntaxWarning: invalid escape sequence '\p'
  descargar_dataframe_como_csv(df_final, "../data\processed/foundational_dataset.csv")
C:\Users\agusm\AppData\Local\Temp\ipykernel_30084\3600581868.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_final['mes'] = df_final['FECHA_HORA'].dt.month
C:\Users\agusm\AppData\Local\Temp\ipykernel_30084\3600581868.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragme